In [1]:
import os
execfile("../script/utils.py")
eventsPath = os.environ["YAHOO_DATA"]
splitedRdd = sc.textFile(eventsPath + "/splitedData")
splitedRdd = splitedRdd.map(parseContextData2)
splitedRdd = sc.parallelize(splitedRdd.take(2000))
#splitedRdd.take(1)
#(uid,[[train],[test]]

In [21]:
def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        # If value has not been encountered yet,
        # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output

def evaluation_rr(testList, recList):
    scores = 0
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                scores = scores + 1.0 / (i+1)
                break
    if len(testList) != 0 :
        scores = scores / len(testList)
    return scores

def evaluation_precision(testItem, recList):
    scores = 0
    if testItem in recList:
        scores = 1.0
    return scores    
    
    

# def evaluation_ap(testList, recList):
#     scores = 0
#     num = 0 
#     rr = []
#     rr[:len(recList)] = [0 for i in range(len(recList))]
#     for t in testList:
#         for i in range(len(recList)):
#             if t == recList[i]:
#                 rr[i] = 1
    
#     for t in testList:    
#         for i in range(len(recList)):
#             if t == recList[i]:
#                 for j in range(i,len(recList)):    
#                     if rr[j] == 1:
#                         scores = scores + 1.0 / (j+1)
#                         num = num + 1
#                 break
#     if num == 0:
#         scores = 0
#     else:
#         scores = scores / num
#     return scores

def evaluation_rap(testList,recList):
    scores = 0.0
    hitNum = 0  
    for i in range(len(recList)):
        if recList[i] in testList:
            hitNum = hitNum + 1
            scores = scores + 1.0*hitNum/(i+1)
    if hitNum != 0:
        scores = scores/hitNum
    else:
        scores = 0
    return scores
        

def evaluation_ap(testList, recList):
    scores = 0
    num = 0 
    rr = []
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    if rr[j] == 1:
                        scores = scores + 1.0 / (j+1)
                        num = num + 1
                break
    if num == 0:
        scores = 0
    else:
        scores = scores / num
    return scores

def evaluation_recall(testList, recList):
    scores = 0
    num = 0
    recall = []
    recall[:len(recList)] = [0 for i in range(len(recList))]
    rr=[]
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    recall[j] = recall[j] + 1.0 
                break    
    recall = [0 if len(testList) == 0 else x/len(testList) for x in recall]
    return recall
def print_result2(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation1.txt','a')
    f.write('2000 users with duplicated evaluations')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[2][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()))
    f.close()
    
def print_result1(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation1.txt','a')
    f.write('2000 users with duplicated evaluations')
    f.write('\n')
    f.write(algorithm_name + ' by precision:'+str(finalScore.mean()))
    f.write('\n')
    f.close()
def print_result2(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation1.txt','a')
    f.write('2000 users with duplicated evaluations')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[2][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()))
            
    f.write(algorithm_name + ' by precision:'+str(finalScore.map(lambda x:x[3]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by rap:'+str(finalScore.map(lambda x:x[4]).mean()))
    f.write('\n')
    f.close()

In [12]:
from collections import Counter
def mfuFunction_P(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    
    for i in range(numTest):  
        #testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender = Counter(RecommenderDuplicate).most_common()
        Recommender =  [t[0] for t in Recommender] 

        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision

        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_precision = scores_precision / numTest

    return scores_precision
finalScore = splitedRdd.map(mfuFunction_P)
print_result1(finalScore,'mfuFunction_P')

In [10]:
from operator import add
numRecList = 5
def lruFunction_P(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    for i in range(numTest):
        trainList = sorted(trainList,key=lambda x:int(x[1][0]), reverse=True);  #sort by timestamp with descending
        testList_items = [t[0] for t in testList]                      #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate

        if len(Recommender) > numRecList-1:
            finalRecommender = Recommender[:numRecList]
        else:    
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender

        scores_precision = evaluation_precision(testList_items[0], finalRecommender) + scores_precision
        trainList = sorted(trainList,key=lambda x:int(x[1][0]));
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_precision = scores_precision / numTest

    return scores_precision

finalScore = splitedRdd.map(lruFunction_P)
#finalScore.map(lambda x:x[2][0]).mean()
#finalScore.take(10)#,finalScore.map(lambda x:x[2]).mean()
print_result1(finalScore, 'lruFunction_P')

In [9]:
def mruFunction_P(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    for i in range(numTest):  
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate

        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_precision = evaluation_precision(testList_items[0], finalRecommender) + scores_precision
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_precision = scores_precision / numTest

    return scores_precision

finalScore = splitedRdd.map(mruFunction_P)
print_result1(finalScore,'mruFunction_P')

In [28]:
from operator import add
numRecList = 5
def lruFunction(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    scores_rap = 0.0
    scores_rr = 0.0
    scores_ap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        trainList = sorted(trainList,key=lambda x:int(x[1][0]), reverse=True);  #sort by timestamp with descending
        testList_items = [t[0] for t in testList]                      #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate

        if len(Recommender) > numRecList-1:
            finalRecommender = Recommender[:numRecList]
        else:    
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        trainList = sorted(trainList,key=lambda x:int(x[1][0]));
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_ap, scores_recall, scores_precision, scores_rap

finalScore = splitedRdd.map(lruFunction)
#finalScore.map(lambda x:x[2][0]).mean()
#finalScore.take(10)#,finalScore.map(lambda x:x[2]).mean()
print_result2(finalScore, 'lru')

In [ ]:
def mruFunction(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_rr = 0.0
    scores_ap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):  
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate

        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        trainList.append(testList[0])
        testList.pop(0)  
        
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]

    return scores_rr, scores_ap, scores_recall
finalScore = splitedRdd.map(mruFunction)
print_result(finalScore,'mru')

In [22]:
from collections import Counter
def mfuFunction(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    scores_rr = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):  
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender = Counter(RecommenderDuplicate).most_common()
        Recommender =  [t[0] for t in Recommender] 

        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_ap, scores_recall, scores_precision, scores_rap
finalScore = splitedRdd.map(mfuFunction)
print_result2(finalScore,'mfu')

In [25]:
def evaluation_rap(testList,recList):
    scores = 0.0
    hitNum = 0  
    for i in range(len(recList)):
        if recList[i] in testList:
            hitNum = hitNum + 1
            scores = scores + 1.0*hitNum/(i+1)
    if hitNum != 0:
        scores = scores/hitNum
    else:
        scores = 0
    return scores

a = [2,3,4,5,1]
b = [1,22,3,14,15]
evaluation_rap(a,b)

0.8333333333333333

In [68]:
from collections import Counter
def mfuFunction_time(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    #trainList = trainList[(numTrain*39/40):numTrain]
#     trainList.extend(testList[:(numTest/2)])
#     numTrain = len(trainList)
#     testList = testList[(numTest/2):]
#     numTest = len(testList)
    trainList.extend(testList)
    numTrain = len(trainList)
    testList = [x for x in trainList if trainList[numTrain-1][1][0] - x[1][0]<=43200]
    numTest = len(testList)
    trainList = trainList[:(numTrain-numTest)]
    scores_precision = 0.0
    scores_rr = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):  
        while (len(trainList)>0 and testList[0][1][0]-trainList[0][1][0]>86400):
            trainList.pop(0)
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender = Counter(RecommenderDuplicate).most_common()
        Recommender =  [t[0] for t in Recommender] 

        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        #trainList.pop(0)
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_ap, scores_recall, scores_precision, scores_rap
finalScore = splitedRdd.map(mfuFunction_time)
print_result2(finalScore,'mfuFunction_time')